# **Dependancies**

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [63]:
PATH_TO_DATA = "data/reg_interval20-23wmand.csv"
data = pd.read_csv(PATH_TO_DATA)

In [64]:
data.head()

,difficulty3rsi,mediantransactionvalue30trxUSD,mining_profitability90trx,price30momUSD,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,top100cap3trx,transactionvalueUSD,price3wmaUSD
0,99.997,-0.240,-0.307,-77.554,28623.0,-231.566,3878.0,-1017.000,573722.0,0.003,12757.0,7192.0
1,100.000,-0.261,-0.308,-191.732,29355.0,-261.045,4729.0,-1046.000,581575.0,-0.020,16735.0,7207.0
2,100.000,-0.279,-0.310,-64.189,29328.0,-42.402,3051.0,-878.841,587195.0,-0.020,23302.0,7300.0
3,100.000,-0.300,-0.311,38.817,29574.0,152.469,11526.0,-626.347,590895.0,-0.010,16928.0,7423.0
4,100.000,-0.325,-0.312,67.278,30437.0,376.272,10731.0,-565.003,593111.0,0.003,14695.0,7527.0


In [65]:
data.shape

(1097, 12)

In [66]:
y = data['price3wmaUSD']

In [67]:
X = data

In [68]:
X['price3wmaUSD'] = X['price3wmaUSD'].shift(1,fill_value=4000)
#X=X[X['priceUSD']!=1]

In [69]:
X

,difficulty3rsi,mediantransactionvalue30trxUSD,mining_profitability90trx,price30momUSD,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,top100cap3trx,transactionvalueUSD,price3wmaUSD
0,99.997,-0.240,-0.307,-77.554,28623.0,-231.566,3878.000,-1017.000,573722.0,0.003,12757.0,4000.0
1,100.000,-0.261,-0.308,-191.732,29355.0,-261.045,4729.000,-1046.000,581575.0,-0.020,16735.0,7192.0
2,100.000,-0.279,-0.310,-64.189,29328.0,-42.402,3051.000,-878.841,587195.0,-0.020,23302.0,7207.0
3,100.000,-0.300,-0.311,38.817,29574.0,152.469,11526.000,-626.347,590895.0,-0.010,16928.0,7300.0
4,100.000,-0.325,-0.312,67.278,30437.0,376.272,10731.000,-565.003,593111.0,0.003,14695.0,7423.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1092,99.043,-0.085,-0.406,425.550,82135.0,-179.890,8243.000,-2758.000,2474152.0,0.031,53190.0,16731.0
1093,99.043,-0.087,-0.405,177.977,76543.0,-258.919,8530.000,-2936.000,2479394.0,0.047,54195.0,16643.0
1094,99.043,-0.090,-0.404,-347.725,82274.0,-257.127,1350.000,-2778.000,2489811.0,0.027,50405.0,16579.0
1095,99.043,-0.094,-0.403,-489.020,86919.0,-69.693,287.999,-2661.000,2500289.0,0.030,42790.0,16567.0


In [70]:
#X1 = data
#X1['target'] = X1['priceUSD'].shift(-1)


In [71]:
#X1.dropna(subset=['target'], inplace=True)

In [72]:
#X = data.iloc[:,:-1]

In [73]:
#y=data.iloc[:,-1:]

In [74]:
#y = data['priceUSD'].shift(-1,fill_value=1)

In [75]:
#y = data['priceUSD']

In [76]:
y

0        7192.0
1        7207.0
2        7300.0
3        7423.0
4        7527.0
         ...   
1092    16643.0
1093    16579.0
1094    16567.0
1095    16561.0
1096    16627.0
Name: price3wmaUSD, Length: 1097, dtype: float64

In [77]:
def create_dataset(dataset, time_step=1):
    dataX = []
    datay = []
    for i in range(len(dataset) - time_step):
        a = dataset.iloc[i:(i + time_step), :].values  # Use .iloc for DataFrame
        dataX.append(a)
        datay.append(dataset.iloc[i + time_step, -1])
    return np.array(dataX), np.array(datay)


In [78]:
#time_step = 15
#X2 ,y2 = create_dataset(X1, time_step)

#print("X: ", X2.shape)
#print("y: ", y2.shape)

In [79]:
#X2

In [80]:
#y2

In [81]:
#y2_reshaped = y2.reshape(-1, 1, 1)

In [82]:
#y2_reshaped

In [83]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [84]:
X_train.shape

(877, 12)

In [85]:
estimators=[]

In [86]:
estimators.append(['robust',RobustScaler()])

In [87]:
estimators.append(['mixmax',MinMaxScaler()])

In [88]:
scale=Pipeline(estimators,verbose=True)

In [89]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [90]:
X_test=scale.transform(X_test)

In [91]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [92]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [93]:
y_train=y_train.values

In [94]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [95]:
y_test=y_test.values

In [96]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [97]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [98]:
X_train.shape

(877, 1, 12)

In [99]:
X_train

array([[[0.99997   , 0.1867065 , 0.03035714, ..., 0.40980393,
         0.        , 0.        ]],

       [[1.        , 0.17102315, 0.02946429, ..., 0.39477125,
         0.00163186, 0.0507674 ]],

       [[1.        , 0.15758029, 0.02767857, ..., 0.39477125,
         0.00432579, 0.05100596]],

       ...,

       [[0.99999   , 0.44660193, 0.03214286, ..., 0.4130719 ,
         0.1255354 , 0.40807953]],

       [[0.00905   , 0.44436148, 0.03035714, ..., 0.43333334,
         0.07643056, 0.40823856]],

       [[0.00283   , 0.44286782, 0.02857143, ..., 0.43986928,
         0.10168917, 0.40609145]]], dtype=float32)

In [100]:
y_train.shape

(877, 1, 1)

In [101]:
y_train

array([[[ 7192.]],

       [[ 7207.]],

       [[ 7300.]],

       [[ 7423.]],

       [[ 7527.]],

       [[ 7756.]],

       [[ 8052.]],

       [[ 8055.]],

       [[ 7993.]],

       [[ 8028.]],

       [[ 8102.]],

       [[ 8144.]],

       [[ 8377.]],

       [[ 8616.]],

       [[ 8707.]],

       [[ 8795.]],

       [[ 8859.]],

       [[ 8877.]],

       [[ 8773.]],

       [[ 8703.]],

       [[ 8682.]],

       [[ 8577.]],

       [[ 8477.]],

       [[ 8390.]],

       [[ 8418.]],

       [[ 8586.]],

       [[ 8850.]],

       [[ 9148.]],

       [[ 9322.]],

       [[ 9373.]],

       [[ 9384.]],

       [[ 9391.]],

       [[ 9376.]],

       [[ 9301.]],

       [[ 9347.]],

       [[ 9532.]],

       [[ 9704.]],

       [[ 9795.]],

       [[ 9948.]],

       [[ 9964.]],

       [[ 9975.]],

       [[10143.]],

       [[10249.]],

       [[10287.]],

       [[10215.]],

       [[10054.]],

       [[ 9859.]],

       [[ 9814.]],

       [[ 9946.]],

       [[ 9810.]],



In [102]:
#X_train, X_test, y_train, y_test =train_test_split(X2,y2_reshaped, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [103]:
#X_train.shape[2]

In [104]:
#y_train.shape

In [105]:
#X_test.shape

# **Model Architecture + Training**

In [106]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [107]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [108]:
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [109]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [110]:
mcp_save = ModelCheckpoint('LSTM_reg_18-21rmse355.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [111]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 30523.2031 - mae: 30523.8945 - val_loss: 20521.5059 - val_mae: 20522.1992
Epoch 2/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 30618.3711 - mae: 30619.0645 - val_loss: 20407.6836 - val_mae: 20408.3770
Epoch 3/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 29852.2207 - mae: 29852.9121 - val_loss: 19885.3145 - val_mae: 19886.0059
Epoch 4/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 29022.7188 - mae: 29023.4141 - val_loss: 18373.4121 - val_mae: 18374.1074
Epoch 5/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 26454.8672 - mae: 26455.5586 - val_loss: 14887.8721 - val_mae: 14888.5654
Epoch 6/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 19967.0195 - mae: 19967.7148 - val_loss: 8897.4404 - val_mae: 8898.1338
Epoch 7/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 13952.8809 - mae: 13953.5732 - val_loss: 4111.8013 - val_mae: 4112.4941
Epoch 8/5000
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss

# **Testing Model**

In [112]:
#model.save('LSTM_complete_model344.h5', overwrite=True)

In [113]:
#from tensorflow.keras.models import load_model
#loaded_model = load_model('LSTM_complete_model344.h5')

In [114]:
y_pred = np.ravel(model.predict(X_test))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [115]:
y_test=np.ravel(y_test)

In [116]:
y_pred

array([29152.518, 28669.176, 28796.99 , 29993.576, 31496.844, 31436.67 ,
       30207.342, 29408.78 , 28957.094, 29343.201, 30425.193, 30123.469,
       30547.621, 30135.346, 30044.691, 29254.254, 28068.555, 25638.773,
       23351.236, 21913.73 , 21802.58 , 21489.658, 20540.686, 19824.5  ,
       20327.771, 21287.814, 21191.602, 21059.762, 21353.854, 21704.303,
       21859.107, 21701.87 , 21443.166, 20867.77 , 20095.705, 19928.174,
       19671.998, 19725.145, 19833.828, 20317.049, 20536.596, 20949.088,
       21828.865, 22022.078, 21643.928, 21052.564, 20226.2  , 20090.258,
       20226.156, 21000.234, 21345.018, 21595.775, 22142.492, 22815.746,
       23710.125, 23724.814, 23712.355, 23015.336, 22945.93 , 22470.537,
       21824.158, 21682.785, 22954.385, 24192.545, 24563.814, 24149.838,
       23752.332, 23321.11 , 23355.549, 23334.996, 23481.203, 23439.564,
       23421.664, 24104.213, 23990.854, 24041.672, 24448.99 , 24480.215,
       24810.76 , 24719.45 , 24834.75 , 24444.027, 

In [117]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9791808290551962

In [118]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

384.67657

In [119]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

494.50046

In [120]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.8345849588513374

In [121]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,384.676575,494.500458,1.834585,0.979181


In [122]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,28988.0,29152.517578
1,29019.0,28669.175781
2,29824.0,28796.990234
3,30909.0,29993.576172
4,31217.0,31496.843750
...,...,...
215,16643.0,16970.142578
216,16579.0,16869.824219
217,16567.0,16798.339844
218,16561.0,16806.830078
